In [1]:
import sys
import os

parent_dir = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(parent_dir)

from utils.process_data import user_item_interaction_scores
from parquet_data_reader import ParquetDataReader
from models.content_based.cosine_model import CosineModel

import polars as pl
import numpy as np
parquet_reader = ParquetDataReader()

In [2]:
train_behavior_df = parquet_reader.read_data("../../data/train/behaviors.parquet")
embeddings_df = parquet_reader.read_data("../../data/document_vector.parquet")
article_df = parquet_reader.read_data("../../data/articles.parquet")
test_behavior_df = parquet_reader.read_data("../../data/validation/behaviors.parquet")

In [3]:
cosine_model = CosineModel(train_behavior=train_behavior_df, embeddings=embeddings_df, articles=article_df)
recommendations = cosine_model.recommend(14241,10)
print(recommendations)

[9773282, 9780193, 9729516, 9739783, 9724453, 9761255, 9591069, 9752312, 9764420, 9774352]


In [ ]:
print(cosine_model.evaluate_recommender(test_behavior_df, 10))